###Lendo as tabelas em parquet

In [0]:
order_details = spark.read.option("inferSchema", "true").parquet('dbfs:/FileStore/tables/Trusted/OrderDetails/part-00000-tid-4985232689994927941-03efc717-a842-4167-a1da-fc84846ba5e1-17-1-c000.snappy.parquet')
products = spark.read.option("inferSchema", "true").parquet('dbfs:/FileStore/tables/Trusted/Products/part-00000-tid-6157483827850255474-16afcc1b-e0ba-4505-802b-0f5e7d6a2717-19-1-c000.snappy.parquet')
categories = spark.read.option("inferSchema", "true").parquet('dbfs:/FileStore/tables/Trusted/Categories/part-00000-tid-1599989917930139817-448bb6b1-804b-45d3-87a2-267e4ecdb437-13-1-c000.snappy.parquet')
suppliers = spark.read.option("inferSchema", "true").parquet('dbfs:/FileStore/tables/Trusted/Suppliers/part-00000-tid-2886166293560326011-c31a72c6-1706-4c0a-8518-019f24994b0b-22-1-c000.snappy.parquet')
 
employees = spark.read.option("inferSchema", "true").parquet('dbfs:/FileStore/tables/Trusted/Employees/part-00000-tid-1974437070719135475-bc42e89a-7804-4ed2-a271-f8066c7b8259-15-1-c000.snappy.parquet')
employee_territories = spark.read.option("inferSchema", "true").parquet('dbfs:/FileStore/tables/Trusted/EmployeeTerritories/part-00000-tid-4786940582130186024-dab66cef-5ca0-4851-940a-61627f21ece7-16-1-c000.snappy.parquet')
territories = spark.read.option("inferSchema", "true").parquet('dbfs:/FileStore/tables/Trusted/Territories/part-00000-tid-4979799626860989691-eef110a6-f937-41ec-8f61-ebb81496ad36-23-1-c000.snappy.parquet')
region = spark.read.option("inferSchema", "true").parquet('dbfs:/FileStore/tables/Trusted/Region/part-00000-tid-2265405182185907058-740f2445-1179-4374-a993-eddc84c91c8c-20-1-c000.snappy.parquet')
 
customers = spark.read.option("inferSchema", "true").parquet('dbfs:/FileStore/tables/Trusted/Customers/part-00000-tid-5224931012289157897-bfd3b7ac-98e0-41b9-9d8d-06932a097d0f-14-1-c000.snappy.parquet')

In [0]:
order_details.printSchema()

root
 |-- OrderID: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)



In [0]:
orders = spark.read.option("inferSchema", "true").parquet('dbfs:/FileStore/tables/Trusted/Orders/part-00000-tid-2810330986264355593-46daf00d-2691-45ce-b52a-46cdf5453a41-18-1-c000.snappy.parquet')

#####Alterando o nome repetido de unitprice

In [0]:
products = products.withColumnRenamed("UnitPrice", "UnitPrice_Prod")
order_details = order_details.withColumnRenamed("UnitPrice", "UnitPrice_OD")

####Realizando os Joins para gerar o Star Schema

In [0]:
df_OrderDetails_fact = order_details.join(products, ['ProductID']) \
                                 .join(categories, ['CategoryID']) \
                                 .join(suppliers, ['SupplierID'])

df_OrderDetails_fact.printSchema()     

root
 |-- SupplierID: string (nullable = true)
 |-- CategoryID: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- OrderID: string (nullable = true)
 |-- UnitPrice_OD: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- QuantityPerUnit: string (nullable = true)
 |-- UnitPrice_Prod: string (nullable = true)
 |-- UnitsInStock: string (nullable = true)
 |-- UnitsOnOrder: string (nullable = true)
 |-- ReorderLevel: string (nullable = true)
 |-- Discontinued: string (nullable = true)
 |-- CategoryName: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Picture: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- ContactName: string (nullable = true)
 |-- ContactTitle: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- PostalCode: string (n

In [0]:
df_Employees_fact = employees.join(employee_territories, employees.Employee == employee_territories.EmployeeID) \
                                   .join(territories, employee_territories.TerritoryID == territories.TerritoryID) \
                                   .join(region, territories.RegionID == region.RegionID)

df_Employees_fact.printSchema()

root
 |-- Employee: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- TitleOfCourtesy: string (nullable = true)
 |-- BirthDate: string (nullable = true)
 |-- HireDate: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- PostalCode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- HomePhone: string (nullable = true)
 |-- Extension: string (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Notes: string (nullable = true)
 |-- ReportsTo: string (nullable = true)
 |-- PhotoPath: string (nullable = true)
 |-- EmployeeID: string (nullable = true)
 |-- TerritoryID: string (nullable = true)
 |-- TerritoryID: string (nullable = true)
 |-- TerritoryDescription: string (nullable = true)
 |-- RegionID: string (nullable = true)
 |-- RegionID: string (nullable = true)
 |-- RegionDescri

In [0]:
df_Customers_fact = customers

df_Customers_fact.printSchema()

root
 |-- CustomerID: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- ContactName: string (nullable = true)
 |-- ContactTitle: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- PostalCode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Fax: string (nullable = true)



####Removendo colunas

In [0]:
colunas_a_manter_Order_Details = ["OrderID","ProductID","Quantity","Discount","ProductName","QuantityPerUnit","UnitPrice_Prod","UnitPrice_OD","UnitsInStock","UnitsOnOrder","ReorderLevel","Discontinued","CategoryName","Description","Picture","CompanyName","ContactName","ContactTitle","Address","City","Region","PostalCode","Country","Phone","Fax","HomePage"]

df_OrderDetails_fact = df_OrderDetails_fact.select(colunas_a_manter_Order_Details)

df_OrderDetails_fact.printSchema()

root
 |-- OrderID: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- QuantityPerUnit: string (nullable = true)
 |-- UnitPrice_Prod: string (nullable = true)
 |-- UnitPrice_OD: string (nullable = true)
 |-- UnitsInStock: string (nullable = true)
 |-- UnitsOnOrder: string (nullable = true)
 |-- ReorderLevel: string (nullable = true)
 |-- Discontinued: string (nullable = true)
 |-- CategoryName: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Picture: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- ContactName: string (nullable = true)
 |-- ContactTitle: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- PostalCode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Phone: string (nullable 

In [0]:
colunas_a_manter_Employees = ["EmployeeID", "Employee", "LastName", "FirstName", "Title", "TitleOfCourtesy", "BirthDate", "HireDate", "Address", "City", "Region", "PostalCode", "Country", "HomePhone", "Extension", "Photo", "Notes", "ReportsTo", "PhotoPath", "TerritoryDescription", "RegionDescription"]

df_Employees_fact = df_Employees_fact.select(colunas_a_manter_Employees)

df_Employees_fact.printSchema()

root
 |-- EmployeeID: string (nullable = true)
 |-- Employee: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- TitleOfCourtesy: string (nullable = true)
 |-- BirthDate: string (nullable = true)
 |-- HireDate: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- PostalCode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- HomePhone: string (nullable = true)
 |-- Extension: string (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Notes: string (nullable = true)
 |-- ReportsTo: string (nullable = true)
 |-- PhotoPath: string (nullable = true)
 |-- TerritoryDescription: string (nullable = true)
 |-- RegionDescription: string (nullable = true)



###Salvando as tabelas em formato Star Schema no Data Warehouse

In [0]:
df_OrderDetails_fact.write.format("parquet").option("header", "true").save("/FileStore/tables/DW/OrderDetails")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-1642855360109065> in <cell line: 1>()
----> 1 df_OrderDetails_fact.write.format("parquet").option("header", "true").save("/FileStore/tables/DW/OrderDetails")

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/readwriter.py in save(self, path, format, mode, partitionBy, **options)
    966             self._jwrite.save()
    967         else:
--> 968             self._jwrite.save(path)
    969 
    970     @since(1.4)

/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway

In [0]:
df_Employees_fact.write.format("parquet").option("header", "true").save("/FileStore/tables/DW/Employees")

In [0]:
df_Customers_fact.write.format("parquet").option("header", "true").save("/FileStore/tables/DW/Customers")

In [0]:
orders.write.format("parquet").option("header", "true").save("/FileStore/tables/DW/Orders_Fact")

####Questão 1 - Quais são os três produtos MENOS vendidos?

In [0]:
import pyspark.sql.functions as f

df_OrderDetails_fact = df_OrderDetails_fact.withColumn("Quantity",f.col("Quantity").cast("int"))

df_OrderDetails_fact.printSchema()

root
 |-- OrderID: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Discount: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- QuantityPerUnit: string (nullable = true)
 |-- UnitPrice_Prod: string (nullable = true)
 |-- UnitPrice_OD: string (nullable = true)
 |-- UnitsInStock: string (nullable = true)
 |-- UnitsOnOrder: string (nullable = true)
 |-- ReorderLevel: string (nullable = true)
 |-- Discontinued: string (nullable = true)
 |-- CategoryName: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Picture: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- ContactName: string (nullable = true)
 |-- ContactTitle: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- PostalCode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Phone: string (nullable

In [0]:
#agrupa os produtos somando a quantidade
df_3menosVend = df_OrderDetails_fact.groupBy('ProductID','ProductName').sum('quantity')
 
#renomeia a coluna de quantidade
df_3menosVend = df_3menosVend.withColumnRenamed('sum(quantity)','TotalComprado')
 
#ordena o dataframe pela quantidade e limita o resultado aos três primeiros
df_3menosVend = df_3menosVend.sort('TotalComprado', ascending=True).limit(3)
 
#mostra o resultado
display(df_3menosVend)

ProductID,ProductName,sum_quantity
9,Mishi Kobe Niku,95
15,Genen Shouyu,122
37,Gravad lax,125


####Questão 2 - Quais são os cinco clientes que MAIS compras fizeram? (quantidade)

In [0]:
#Alterando os nomes duplicados de ContactName
df_Customers_fact = df_Customers_fact.withColumnRenamed("ContactName", "Customer_ContactName")
df_OrderDetails_fact = df_OrderDetails_fact.withColumnRenamed("ContactName", "Supplier_ContactName")

#Realizando o merge entre Orders, a dimensão Customers e OrderDetails
df_Orders_Customers = df_Customers_fact.join(orders, "CustomerID") \
    .join(df_OrderDetails_fact, "OrderID")

#agrupa os produtos somando a quantidade
df_5maioresCli = df_Orders_Customers.groupBy('Customer_ContactName').sum('quantity')
 
#renomeia a coluna de quantidade
df_5maioresCli = df_5maioresCli.withColumnRenamed('sum(quantity)','TotalComprado')
 
#ordena o dataframe pela quantidade e limita o resultado aos três primeiros
df_5maioresCli = df_5maioresCli.sort('TotalComprado', ascending=False).limit(5)
 
#mostra o resultado
display(df_5maioresCli)

Customer_ContactName,TotalComprado
Jose Pavarotti,4958
Roland Mendel,4543
Horst Kloss,3961
Patricia McKenna,1684
Peter Franken,1525


####Questão 3 - Quais são os cinco clientes com MAIOR total gasto? (montante)

In [0]:
from pyspark.sql.functions import sum, expr

#agrupa os produtos somando a quantidade
df_5maioresGastos = df_Orders_Customers.groupBy('Customer_ContactName').agg(expr("sum(Quantity * UnitPrice_OD * (1 - Discount))"))
 
#renomeia a coluna de quantidade
df_5maioresGastos = df_5maioresGastos.withColumnRenamed('sum(((Quantity * UnitPrice_OD) * (1 - Discount)))','TotalGasto')
 
#ordena o dataframe pela quantidade e limita o resultado aos três primeiros
df_5maioresGastos = df_5maioresGastos.sort('TotalGasto', ascending=False).limit(5)
 
#mostra o resultado
display(df_5maioresGastos)

Customer_ContactName,TotalGasto
Roland Mendel,28163.0
Jose Pavarotti,26887.0
Horst Kloss,17983.0
Maria Larsson,14071.0
Philip Cramer,10107.0


In [0]:
#Versão Marcelo
total_gasto_por_cliente = df_Orders_Customers.groupBy("Customer_ContactName") \
    .agg(expr("sum(Quantity * UnitPrice_OD * (1 - Discount))").alias("TotalGasto")) \
    .orderBy("TotalGasto", ascending=False)

cinco_maior_gasto = total_gasto_por_cliente.limit(5)
cinco_maior_gasto.show()

+--------------------+----------+
|Customer_ContactName|TotalGasto|
+--------------------+----------+
|       Roland Mendel|   28163.0|
|      Jose Pavarotti|   26887.0|
|         Horst Kloss|   17983.0|
|       Maria Larsson|   14071.0|
|       Philip Cramer|   10107.0|
+--------------------+----------+

